In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.csv as csv
import datetime
import folium
import time
import geopandas

In [2]:
data=pd.read_csv('../../../data/stop_activity_granular_2020-09-01_2020-10-31.csv', engine='pyarrow')

In [3]:
df=data.copy()

In [4]:
len(df)

22506627

In [5]:
df.head()

,OPERATION_DATE,BOOKING_ID,SCHED_DAY_TYPE_CODED_NUM,SERVICE_RTE_LIST,TRIP_COMPASS_DIR_CD,KEY_BLOCK_NUM,STOP_SEQUENCE_NUM,STOP_ID,STOP_NM,SCHED_ARRIVAL_SECS_AFTER_MIDNT,FULL_DATE,ACTUAL_ARRIVAL_SECS_AFTER_MIDNT,ACTUAL_DEPARTURE_SECS_AFTER_MIDNT,PSNGR_BOARDINGS,PSNGR_ALIGHTINGS,TRIP_ID,DEP_PSNGR_LOAD
0,2020-09-01,SUM20,0,65,North,6503,13.0,10020,NE 55TH ST & 37TH AVE NE,NaN,09/01/2020,110709,110709,0,0,46931260,2
1,2020-09-01,SUM20,0,120,North,12011,32.0,21560,DELRIDGE WAY SW & SW HUDSON ST,NaN,09/01/2020,123209,123209,0,0,47583182,0
2,2020-09-01,SUM20,0,107,NULL,10714,0.0,41890,S LANDER ST & 15TH AVE S,155200.0,09/01/2020,155345,155345,0,0,47801614,0
3,2020-09-01,SUM20,0,120,NULL,12024,1.0,98141,2ND AVE & LENORA ST,124400.0,09/01/2020,124003,124003,0,0,47583379,0
4,2020-09-01,SUM20,0,671,South,67111,5.0,61100,INTERNATIONAL BLVD & S 188TH ST,NaN,09/01/2020,133751,133818,4,0,47799588,5


In [6]:
len(set(df['FULL_DATE']))

60

In [7]:
len(set(df['STOP_ID']))

5993

In [8]:
len(set(df['SERVICE_RTE_LIST']))

152

In [9]:
len(set(df['TRIP_ID']))

82027

In [10]:
df['PSNGR_ALIGHTINGS'].describe()

count    2.250663e+07
mean     2.784130e-01
std      9.474394e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.080000e+02
Name: PSNGR_ALIGHTINGS, dtype: float64

In [11]:
df.isnull().sum()

OPERATION_DATE                              0
BOOKING_ID                                  0
SCHED_DAY_TYPE_CODED_NUM                    0
SERVICE_RTE_LIST                            0
TRIP_COMPASS_DIR_CD                         0
KEY_BLOCK_NUM                               0
STOP_SEQUENCE_NUM                        2441
STOP_ID                                     0
STOP_NM                                     0
SCHED_ARRIVAL_SECS_AFTER_MIDNT       17238389
FULL_DATE                                   0
ACTUAL_ARRIVAL_SECS_AFTER_MIDNT             0
ACTUAL_DEPARTURE_SECS_AFTER_MIDNT           0
PSNGR_BOARDINGS                             0
PSNGR_ALIGHTINGS                            0
TRIP_ID                                     0
DEP_PSNGR_LOAD                              0
dtype: int64

In [12]:
m_start=datetime.time(6,30,0)
m_end=datetime.time(9,0,0)
e_start=datetime.time(17,0,0)
e_end=datetime.time(19,0,0)

In [13]:
df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']=pd.to_datetime(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT'],unit='s')
df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']=df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT'].dt.time

In [40]:
df_m_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>m_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<m_end)]
df_m_rush

,OPERATION_DATE,BOOKING_ID,SCHED_DAY_TYPE_CODED_NUM,SERVICE_RTE_LIST,TRIP_COMPASS_DIR_CD,KEY_BLOCK_NUM,STOP_SEQUENCE_NUM,STOP_ID,STOP_NM,SCHED_ARRIVAL_SECS_AFTER_MIDNT,FULL_DATE,ACTUAL_ARRIVAL_SECS_AFTER_MIDNT,ACTUAL_DEPARTURE_SECS_AFTER_MIDNT,PSNGR_BOARDINGS,PSNGR_ALIGHTINGS,TRIP_ID,DEP_PSNGR_LOAD
0,2020-09-01,SUM20,0,65,North,6503,13.0,10020,NE 55TH ST & 37TH AVE NE,NaN,09/01/2020,110709,06:45:09,0,0,46931260,2
23,2020-09-01,SUM20,0,156,West,15605,2.0,79960,53RD AVE S & KLICKITAT DR,NaN,09/01/2020,112901,07:21:41,0,0,47800135,1
32,2020-09-01,SUM20,0,73,South,7303,17.0,38022,UNIVERSITY WAY NE & NE RAVENNA BLVD,NaN,09/01/2020,113059,07:24:19,0,0,42019428,0
42,2020-09-01,SUM20,0,65,South,6515,18.0,24930,35TH AVE NE & 34TH AVE NE,NaN,09/01/2020,113331,07:28:51,0,0,44052451,0
51,2020-09-01,SUM20,0,45,West,4507,4.0,9561,NE PACIFIC ST & UNIVERSITY WAY NE,NaN,09/01/2020,113831,07:37:11,0,0,47377561,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22506603,2020-10-31,FAL20,1,50,East,5006,32.0,99253,SODO BUSWAY & S SPOKANE ST,NaN,10/31/2020,200948,07:49:08,0,0,47804738,2
22506606,2020-10-31,FAL20,1,50,East,5006,31.0,99263,SODO BUSWAY & S LANDER ST,200900.0,10/31/2020,200752,07:46:48,0,0,47804738,2
22506619,2020-10-31,FAL20,1,183,NULL,18302,0.0,99804,FEDERAL WAY TC,114200.0,10/31/2020,114007,07:40:07,0,0,49465592,0
22506620,2020-10-31,FAL20,1,554,NULL,55402,0.0,99811,HIGHLANDS DR NE,201400.0,10/31/2020,200929,07:48:49,0,0,41378534,0


In [42]:
# show stops with the most average boardings during the morning rush hours
df_top_morning=df_morning.nlargest(10,'PSNGR_BOARDINGS')
df_top_morning

,STOP_ID,PSNGR_BOARDINGS,STOP_NM
1235,21740,4.305466,S HOLGATE ST & AIRPORT WAY S
68,1180,3.759630,PIKE ST & 4TH AVE
99,1560,3.484401,JAMES ST & 3RD AVE
845,13721,3.430928,15TH AVE NW & NW MARKET ST
3353,59314,3.376190,ANDOVER PARK W & BAKER BLVD BAY #3
29,578,3.075432,3RD AVE & PIKE ST
915,16106,3.072072,AURORA VILLAGE TRANSIT CENTER BAY #6
2283,41955,3.000000,9TH AVE & SPRING ST
791,12880,2.958032,JEFFERSON ST & 9TH AVE
3140,57451,2.893424,KENT SOUNDER STATION BAY #1


In [118]:
# covert 'OPERATION_DATE' to datetime and initialize a reinstatement date
df['OPERATION_DATE']=pd.to_datetime(df['OPERATION_DATE'])
reinstatement_date = pd.to_datetime("2020-10-01").date()

# filter into 2 datasets: before and after
before_df = df_m_rush.loc[(df_m_rush['OPERATION_DATE'] <= reinstatement_date)]
after_df = df_m_rush.loc[(df_m_rush['OPERATION_DATE'] > reinstatement_date)]

In [90]:
def load_geoid_stops_matchings():
    """Load GEOID stops and matchings."""
    df=pd.read_csv('../../../data/bus_stop_geoid_matching.csv', engine='pyarrow')
    df['GEOID'] = df['GEOID'].astype('Int64').astype('str')
    df['STOP_ID']=df['stop_id']
    df=df[['STOP_ID', 'GEOID']]
    return df

df_stop_geoid_matchings = load_geoid_stops_matchings()
df_stop_geoid_matchings

,STOP_ID,GEOID
0,3169,53033024902
1,3170,53033023300
2,3171,53033023300
3,3172,53033023300
4,3173,53033030314
...,...,...
14509,28959,53033003100
14510,28960,53033003100
14511,28961,53033003100
14512,28962,53033003100


In [124]:
def add_GEOID(df):
    """
    Add GEOID to any dataframe, matching on 'variable STOP_ID'
    """
    ret_df=df.merge(df_stop_geoid_matchings, on='STOP_ID')
    return ret_df

def agg_by_GEOID(df):
    """
    Aggregate bus stops by GEOID. Adds GEOID using add_GEOID(), and aggregates by those GEOID codes
    
    df must have 'STOP_ID' and 'PSNGR_BOARDINGS'
    """
    df_ret = df.groupby('STOP_ID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).reset_index()
    df_ret = add_GEOID(df_ret)
    df_ret.groupby('GEOID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).reset_index()
    return df_ret

In [151]:
before_df_agg = agg_by_GEOID(before_df)
after_df_agg = agg_by_GEOID(after_df)

In [150]:
print(f'The two bus stops that have data before but not after: {set(before_df_agg.STOP_ID) - set(after_df_agg.STOP_ID)}')

The two bus stops that have data before but not after: {17280, 10654}


In [152]:
before_df_agg.describe()

,STOP_ID,PSNGR_BOARDINGS
count,788.000000,788.000000
mean,12049.513959,0.259261
std,6316.519031,0.413479
min,3190.000000,0.000000
25%,7315.000000,0.036260
50%,10651.500000,0.108454
75%,16102.250000,0.319182
max,28680.000000,4.348101


In [153]:
after_df_agg.describe()

,STOP_ID,PSNGR_BOARDINGS
count,786.000000,786.000000
mean,12044.634860,0.242260
std,6321.606526,0.395848
min,3190.000000,0.000000
25%,7245.000000,0.026421
50%,10645.000000,0.097864
75%,16101.500000,0.295915
max,28680.000000,4.261438


In [154]:
0.259261 - 0.242260

0.017001000000000016

The mean number of passenger boardings during morning rush hours of 6:30-9 AM decreased after fare reinstatement. In the month of September 2020, there was an average of 0.259 boardings per stop in each census tract. In the month of October 2020, after the reinstatement date of October 1st, there was an average of 0.242 boardings per stop in each census tract.

This is for boardings during morning hours specifically - I think looking at boardings during rush hours specifically will be useful for when we look at location-specific data, because I’m assuming that boardings in morning rush hours usually indicates that folks live in the census tract at those bus stops.

One thing I think we will have to investigate or add to our analysis was the fact that routes were changed during this time - perhaps we can look at the cumulative number of boardings at each bus stop as opposed to average to account for the route changes just to see the # of unique individuals using the bus

# Old Stuff

In [ ]:
df['PSNGR_ACTIVITY']=df['PSNGR_ALIGHTINGS']+df['PSNGR_BOARDINGS']

In [ ]:
df['FULL_DATE']=pd.to_datetime(df['FULL_DATE'])

Looking for most popular stops for morning and evening rush hours. To see where people commute to and from work, this should align with data on work tract and household tract. 

Where do we want to focus on


In [ ]:
df.value_counts('OPERATION_DATE').plot()

In [ ]:
df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']

In [ ]:
top_morning

In [ ]:
geoloc=pd.read_csv('FinalData/raw_data/1_gtfs/stops.txt')
geoloc['geometry'] = geopandas.points_from_xy(geoloc['stop_lon'], geoloc['stop_lat'])

In [ ]:
morning=geoloc.merge(top_morning, how='inner', right_on='STOP_ID',left_on="stop_id")
morning

In [ ]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in morning.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "Stamen Terrain", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(morning.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

In [ ]:
mapping

Repeating the same process for evening rush hours for the first date

In [ ]:
df_e_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>e_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<e_end)]

In [ ]:
top_evening=df_e_rush.loc[(df_e_rush['OPERATION_DATE'] == '2020-09-01')].groupby('STOP_ID').agg({'PSNGR_ACTIVITY':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_ACTIVITY')

In [ ]:
evening=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")
evening

In [ ]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in evening.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

In [ ]:
mapping

In [ ]:
df_e_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>e_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<e_end)]

top_evening=df_e_rush.groupby('STOP_ID').agg({'PSNGR_ACTIVITY':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_ACTIVITY')

evening=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")
evening

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in evening.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

mapping

In [ ]:
df_e_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>e_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<e_end)]

top_evening=df_e_rush.loc[(df_e_rush['OPERATION_DATE'] == '2020-09-01')].groupby('STOP_ID').agg({'PSNGR_ALIGHTINGS':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_ALIGHTINGS')

evening=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")
evening

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in evening.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

mapping

In [ ]:
df_e_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>e_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<e_end)]

top_evening=df_e_rush.groupby('STOP_ID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_BOARDINGS')

evening=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")
evening

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in evening.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

mapping

In [ ]:
df_m_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>m_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<m_end)]

top_evening=df_m_rush.loc[(df_m_rush['OPERATION_DATE'] == '2020-09-01')].groupby('STOP_ID').agg({'PSNGR_ALIGHTINGS':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_ALIGHTINGS')

morning=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")


geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in morning.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

mapping

In [ ]:
df_m_rush=df[(df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']>m_start) & (df['ACTUAL_DEPARTURE_SECS_AFTER_MIDNT']<m_end)]

top_evening=df_m_rush.loc[(df_m_rush['OPERATION_DATE'] == '2020-09-01')].groupby('STOP_ID').agg({'PSNGR_BOARDINGS':'mean','STOP_NM':'first'}).nlargest(10,'PSNGR_BOARDINGS')

morning=geoloc.merge(top_evening, how='inner', right_on='STOP_ID',left_on="stop_id")


geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in morning.geometry ]
mapping = folium.Map(location = [47.59789702474227, -122.31735261797544], tiles = "OpenStreetMap", zoom_start = 10)
i=0
for coordinates in geo_df_list[:10]:
    mapping.add_child(folium.Marker(location = coordinates,popup =
                            "Name: " + str(evening.stop_name[i]) + '<br>',
                            icon = folium.Icon()))
    i = i + 1

mapping